# 导入附件2：电视产品信息

In [1]:
import pandas as pd
import re
pd.set_option('display.max_columns', None)

In [2]:
data_2 = "附件2：电视产品信息数据.csv"
tv_prod = pd.read_csv(data_2, encoding= 'gb18030')

In [3]:
wu_index = tv_prod[tv_prod["连续剧分类"] == "无"].index
tv_prod.loc[wu_index, "连续剧分类"] = tv_prod.loc[wu_index, "正题名"]
tv_prod.rename(columns= {"连续剧分类":"节目名称"}, inplace=True)


def clean_by_pattern(df):
    pattern = "^\d+\-\d+(\w+)?"
    df[pattern] = [re.findall(pattern, i) for i in df["节目名称"]]
    df[pattern+"_true"] = [len(i)!=0 for i in df[pattern]]
    df.loc[df[pattern+"_true"], "节目名称"] = [i[0] for i in df.loc[df[pattern+"_true"], pattern]]
    df.drop([pattern, "^\d+\-\d+(\w+)?_true"], axis=1, inplace=True)
    return df

def clean_program_name(df):
    '''
    清洗节目名称的函数
    '''
    
    key_words = ["甜果学堂", "花城全视角", "优·秀", "体育竞技广告", "甜果屋", "今天我上线", "兔智来了", "神奇萝卜", "舌尖上的中国Ⅱ", 
             "开心超人联盟", "舌尖上的中国", "星际小蚂蚁", "小小羊提米", "食得健康吗", "阿U", "经典广告", "爆裂飞车", "赌城群英会", 
             "综艺娱乐广告", "维多利亚的秘密", "开心球", "食得健康吗", "大鹏嘚吧嘚", "黑子的篮球", "斗龙战士4", "西游记", "西游记续", 
            "生肖传奇", "那年花开月正圆", "家庭影院广告", "频道一广告", "宣传频道广告", "野外独游", "改变世界的武器", "厝边头尾", 
            "圣龙的守护", "启源星之战", "战神崛起", "最终幻想", "头号公敌", "赛尔号4", "贪狼",
            "关晓彤生日会", "CP","170415平安365", "电视剧场广告", "我的电视广告"]
    
    
    # 去掉名称前后的括号() 
    df["节目名称"] = [re.sub("\(\S*?\)+?", "", i) for i in df["节目名称"]]
    df["节目名称"] = [re.sub("\(.*", "", i) for i in df["节目名称"]]
    df["节目名称"] = [re.sub("（\S*?）+?", "", i) for i in df["节目名称"]]
    df["节目名称"] = [re.sub("（\S*?\)+?", "", i) for i in df["节目名称"]]
    # 去掉名称前的日期 
    for i in df["节目名称"].index: 
        if re.findall(r"月(.+)日 ?(.+)", df.loc[i, "节目名称"]): 
                df.loc[i, "节目名称"] = re.search(r"月(.+)日 ?(.+)", df.loc[i, "节目名称"]).group(2) 
        if "】" in df.loc[i, "节目名称"]: 
            df.loc[i, "节目名称"] = df.loc[i, "节目名称"].split("】")[1]
        if "：" in df.loc[i, "节目名称"]:
            df.loc[i, "节目名称"] = df.loc[i, "节目名称"].split("：")[0]
        if " "  in df.loc[i, "节目名称"]:  
            df.loc[i, "节目名称"] = df.loc[i, "节目名称"].split(" ")[0]
        
        if "《" in df.loc[i, "节目名称"]: 
                df.loc[i, "节目名称"] = re.search(r"《(.+)》?(.+)", df.loc[i, "节目名称"]).group(1) 
        
        for key_word in key_words:
            if key_word in df.loc[i, "节目名称"]:
                df.loc[i, "节目名称"] = key_word
    
    df = clean_by_pattern(df)
    df.drop(df.loc[df["节目名称"].str.contains("测试")].index, inplace=True)

In [5]:
#清洗产品名称
clean_program_name(tv_prod)

# 对电视产品的形式进行分类

In [7]:
#形式：电视剧
play_index = tv_prod["分类名称"].str.contains("电视剧场")
tv_prod.loc[play_index, "形式"] = "电视剧"

#形式：电影
film_index = tv_prod["分类名称"].str.contains("家庭影院")
tv_prod.loc[film_index, "形式"] = "电影"

#形式：综艺
variety_index = tv_prod["分类名称"].str.contains("综艺娱乐")
tv_prod.loc[variety_index, "形式"] = "综艺"

#形式：体育竞技
sport_index = tv_prod["分类名称"].str.contains("体育竞技")
tv_prod.loc[sport_index, "形式"] = "体育竞技"

#形式：新闻时事
news_index = tv_prod["分类名称"].str.contains("新闻时事")
tv_prod.loc[news_index, "形式"] = "新闻时事"
tv_prod.loc[tv_prod["节目名称"].str.contains("G4"), "形式"] = "新闻时事"

#形式：音乐
tv_prod.loc[tv_prod["批次"]=="卡拉OK", "形式"] = "音乐"
tv_prod.loc[tv_prod["节目名称"].str.contains("CP"), "形式"] = "音乐"
tv_prod.loc[tv_prod["节目名称"].str.contains("演唱会"), "形式"] = "音乐"
tv_prod.loc[tv_prod["节目名称"].str.contains("音悦V榜"), "形式"] = "音乐"
tv_prod.loc[tv_prod["节目名称"].str.contains("八音汇"), "形式"] = "音乐"

#形式：广告
ad_index = tv_prod["分类名称"].str.contains("广告")
tv_prod.loc[ad_index, "形式"] = "广告"
tv_prod.loc[tv_prod["节目名称"].str.contains("抢先看"), "形式"] = "广告"

#形式：科学教育
science_edu_index = tv_prod["分类名称"].str.contains("科学教育")
tv_prod.loc[science_edu_index, "形式"] = "科学教育"

#形式：生活服务
life_service_index = tv_prod["分类名称"].str.contains("生活服务")
tv_prod.loc[life_service_index, "形式"] = "生活服务"
tv_prod.loc[tv_prod["节目名称"].str.contains("健康"), "形式"] = "生活服务"


# 对电视产品的类型进行分类

In [8]:
#音乐类型：MV\演唱会
tv_prod.loc[tv_prod["批次"]=="卡拉OK", "类型"] = "MV"
tv_prod.loc[tv_prod["节目名称"].str.contains("CP"), "类型"] = "MV"
tv_prod.loc[tv_prod["节目名称"].str.contains("音悦V榜"), "类型"] = "MV"
tv_prod.loc[tv_prod["节目名称"].str.contains("八音汇"), "类型"] = "MV"
tv_prod.loc[tv_prod["节目名称"].str.contains("演唱会"), "类型"] = "Live"
tv_prod.loc[tv_prod["节目名称"].str.contains("音乐会"), "类型"] = "Live"


#新闻类型：央视新闻|地方新闻\资讯
tv_prod.loc[tv_prod["内容描述"].str.contains("新闻联播"), "类型"] = "央视新闻" 
tv_prod.loc[(tv_prod["分类名称"]=="新闻时事") & (tv_prod["类型"].isnull()), "类型"] = "地方新闻"
tv_prod.loc[tv_prod["节目名称"].str.contains("G4"), "类型"] = "资讯"


#广告类型：宣传广告|电视剧广告|电影广告|综艺广告|体育广告
tv_prod.loc[tv_prod["分类名称"] == "广告", "类型"] = "宣传广告"
tv_prod.loc[tv_prod["节目名称"].str.contains("电视"), "类型"] = "电视剧广告" 
tv_prod.loc[tv_prod["节目名称"].str.contains("影院"), "类型"] = "电影广告" 
tv_prod.loc[tv_prod["节目名称"].str.contains("大片"), "类型"] = "电影广告"
tv_prod.loc[tv_prod["节目名称"].str.contains("抢先看"), "类型"] = "电影广告"
tv_prod.loc[tv_prod["节目名称"].str.contains("综艺"), "类型"] = "综艺广告" 
tv_prod.loc[tv_prod["节目名称"].str.contains("体育"), "类型"] = "体育广告"


#体育竞技类型：其他运动|足球|篮球|羽毛球|乒乓球|围棋|垂钓|高尔夫
tv_prod.loc[tv_prod["分类名称"]=="体育竞技", "类型"] = "其他运动"
tv_prod.loc[tv_prod["节目名称"].str.contains("围棋"), "类型"] = "高尔夫"
tv_prod.loc[tv_prod["节目名称"].str.contains("围棋"), "类型"] = "围棋"
tv_prod.loc[tv_prod["节目名称"].str.contains("钓"), "类型"] = "垂钓"
tv_prod.loc[tv_prod["节目名称"].str.contains("VS"), "类型"] = "足球"
tv_prod.loc[tv_prod["节目名称"].str.contains("足"), "类型"] = "足球"
tv_prod.loc[tv_prod["节目名称"].str.contains("篮"), "类型"] = "篮球"
tv_prod.loc[tv_prod["节目名称"].str.contains("BA"), "类型"] = "篮球"
tv_prod.loc[tv_prod["节目名称"].str.contains("羽"), "类型"] = "羽毛球"
tv_prod.loc[tv_prod["节目名称"].str.contains("乒"), "类型"] = "乒乓球"


#生活服务类型：健康|生活
for i in ["养生", "健康", "健身"]:
    health_index = tv_prod["节目名称"].str.contains(i)
    tv_prod.loc[health_index, "类型"] = "健康"
tv_prod.loc[(tv_prod["分类名称"] == "生活服务") & (tv_prod["类型"].isnull()), "类型"] = "生活"
tv_prod.loc[tv_prod["节目名称"].str.contains("甜果学堂"), "类型"] = "生活"


#科学教育类别：纪录|少儿
tv_prod.loc[(tv_prod["分类名称"]=="科学教育") & (tv_prod["类型"].isnull()), "类型"] = "纪录"
tv_prod.loc[(tv_prod["分类名称"]=="科学教育") & (tv_prod["编码码率"]=="2M"), "类型"] = "少儿"
tv_prod.loc[tv_prod["分类名称"].str.contains("纪录"), "类型"] = "纪录"


tv_prod.loc[tv_prod["类型"] =="生活", "形式"] = "生活服务"
tv_prod.loc[tv_prod["类型"] =="健康", "形式"] = "生活服务"
tv_prod.loc[tv_prod["类型"] =="足球", "形式"] = "体育竞技"



for i in ["语文", "数学", "英语"]:
    tv_prod.loc[tv_prod["节目名称"].str.contains(i), "类型"] = "少儿"
    tv_prod.loc[tv_prod["节目名称"].str.contains(i), "形式"] = "科学教育"
    
for i in ["儿歌","启蒙", "成长"]:
    tv_prod.loc[(tv_prod["形式"] =="生活服务") & (tv_prod["类型"] =="健康") &
                (tv_prod["节目名称"].str.contains(i)), "类型"] = "少儿"
    tv_prod.loc[(tv_prod["形式"] =="生活服务") & (tv_prod["类型"] =="少儿") &
                (tv_prod["节目名称"].str.contains(i)), "形式"] = "科学教育"

In [9]:
#只保留节目名称，声道语种，地区参数，形式，类型这几个特征
tv_prod = tv_prod.loc[:,["节目名称", "声道语种", "地区参数","形式","类型"]].drop_duplicates()
tv_prod.reset_index(drop=True, inplace=True)

In [12]:
tv_prod["类型"].value_counts()

纪录       67
宣传广告     34
足球       21
其他运动     20
生活       19
MV       12
Live     10
健康        9
乒乓球       9
篮球        9
少儿        8
电影广告      7
羽毛球       3
地方新闻      3
央视新闻      2
电视剧广告     2
资讯        1
综艺广告      1
体育广告      1
Name: 类型, dtype: int64

In [11]:
tv_prod["形式"].value_counts()

电视剧     834
电影      242
综艺      173
科学教育     74
体育竞技     60
广告       45
生活服务     28
音乐       20
新闻时事      6
Name: 形式, dtype: int64

In [18]:
#查看除了电视剧、电影、综艺以外，类型是否有缺省值
tv_prod.loc[tv_prod["形式"] == "新闻时事", "类型"].value_counts()

地方新闻    3
央视新闻    2
资讯      1
Name: 类型, dtype: int64

In [19]:
#tv_prod.loc[(tv_prod["形式"] == "新闻时事") &(tv_prod["类型"].isnull())]

,节目名称,声道语种,地区参数,形式,类型


# 将电视产品分成两部分

In [27]:
#电视剧，电影，综艺
tfv_prod = tv_prod.loc[(tv_prod["形式"]=="电视剧")|(tv_prod["形式"]=="电影")|(tv_prod["形式"]=="综艺")]
tfv_prod.reset_index(drop=True, inplace=True)

#剩余电视产品
rest_prod = tv_prod.drop(tv_prod.loc[(tv_prod["形式"]=="电视剧")|(tv_prod["形式"]=="电影")|(tv_prod["形式"]=="综艺")].index)
rest_prod.reset_index(drop=True, inplace=True)

tfv_prod_list = tfv_prod["节目名称"].unique().tolist()

# #判断是否有新的电视产品
# program = pd.read_csv("all_program.csv", encoding="GB18030")
# new_prod = []
# tfv = list(set(tfv_prod_list))
# for i in tfv:
#     if i not in list(program["节目名称"]):
#         new_prod.append(i)

# 爬虫：获取tfv的类型，地区，声道

In [28]:
import requests
from lxml import etree



# 代理服务器
proxyHost = "http-cla.abuyun.com"
proxyPort = "9030"

# 代理隧道验证信息
proxyUser = "H8G8262307V727CC"
proxyPass = "0C9770B8F66C9CA5"

proxyMeta = "http://%(user)s:%(pass)s@%(host)s:%(port)s" % {
      "host" : proxyHost,
      "port" : proxyPort,
      "user" : proxyUser,
      "pass" : proxyPass,
    }

proxies = {
     
        "https" : proxyMeta
    }

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36"
           
          }

def parse_url(url):
    try:
        r = requests.get(url, headers=headers, timeout =20)
        r.raise_for_status()
        response = r.content.decode()
        html = etree.HTML(response)
        return html
    except:
        print("error")

def get_tfv_info(name):
    name = name
    url = "https://www.douban.com/search?cat=1002&q={}".format(name)
    name_html = parse_url(url)
    name_url = name_html.xpath("//*[@id='content']/div/div[1]/div[3]/div[2]/div[1]/div[2]/div/h3/a/@href")
    form = name_html.xpath("//*[@id='content']/div/div[1]/div[3]/div[2]/div[1]/div[2]/div/h3/span[1]/text()")
    
    tfv ={}
    tfv["节目名称"] = name
    #形式
    if form:
        tfv["形式"] = form[0][1:-1]
    else:
        tfv["形式"] = ""
        
    
    try:
        info_html = parse_url(name_url[0])
        lst_temp = info_html.xpath("//*[@id='info']/text()")
        str_temp = "".join(lst_temp).strip()
        tags = info_html.xpath("//div[@id='info']/span[@property='v:genre']/text()")
        tfv["类型"] = tags
        tfv["地区参数"] = re.search("[日本香港台湾中韩美法英]+国?", str_temp).group(0)
        tfv["声道语种"] = re.search("[汉日英粤韩法]语+", str_temp).group(0)
    except:
        tfv["类型"] = ""
        tfv["地区参数"] = "其他地区"
        tfv["声道语种"] = "其他"
    
    tfv_list.append(tfv)
    print(tfv)

In [29]:
import time
from multiprocessing.dummy import Pool as ThreadPool

tfv_list = []
start = time.time()
pool = ThreadPool(3)  #设置并发数量
pool.map(get_tfv_info,tfv_prod_list)
pool.close() 
pool.join()
end = time.time()
print("用时：",end-start)

{'形式': '电视剧', '类型': ['剧情', '古装'], '地区参数': '中国', '节目名称': '通天狄仁杰', '声道语种': '汉语'}
{'形式': '电影', '类型': ['喜剧', '动画', '家庭', '奇幻', '冒险'], '地区参数': '中国', '节目名称': '喜羊羊与灰太狼之嘻哈闯世界', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['剧情', '动画', '奇幻', '武侠', '古装'], '地区参数': '中国', '节目名称': '太乙仙魔录之灵飞纪', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['喜剧', '爱情'], '地区参数': '中国', '节目名称': '漂亮的李慧珍', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['剧情', '爱情'], '地区参数': '中国', '节目名称': '职场是个技术活', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['剧情'], '地区参数': '中国', '节目名称': '深海利剑', '声道语种': '汉语'}
{'形式': '电影', '类型': ['喜剧', '动画', '奇幻'], '地区参数': '中国', '节目名称': '猪猪侠之拼装特工队', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['脱口秀'], '地区参数': '中国', '节目名称': '金星秀', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['喜剧'], '地区参数': '香港', '节目名称': '财神驾到', '声道语种': '粤语'}
{'形式': '电影', '类型': ['剧情', '战争'], '地区参数': '香港', '节目名称': '爱人同志', '声道语种': '粤语'}
{'形式': '电视剧', '类型': ['剧情', '爱情'], '地区参数': '中国', '节目名称': '赛小花的远大前程', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['喜剧', '动画'], '地区参数': '中国', '节目名称': '熊出没之丛林总动员', '声道语种': '汉语'}
{'形式': '电

{'形式': '电视剧', '类型': ['悬疑', '犯罪'], '地区参数': '中国', '节目名称': '零号国境线', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['动画', '儿童'], '地区参数': '中国', '节目名称': '魔力课堂之踢踢和奇奇', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['动画'], '地区参数': '中国', '节目名称': '小龙大功夫', '声道语种': '汉语'}
{'形式': '电影', '类型': ['喜剧'], '地区参数': '美国', '节目名称': '热恋嘉年华', '声道语种': '英语'}
{'形式': '电视剧', '类型': ['剧情', '喜剧'], '地区参数': '美国', '节目名称': '我家也有大明星', '声道语种': '英语'}
{'形式': '电视剧', '类型': ['动画', '儿童', '奇幻', '冒险'], '地区参数': '中国', '节目名称': '电脑总动员', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['剧情', '爱情'], '地区参数': '台湾', '节目名称': '回到爱以前', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['剧情'], '地区参数': '韩国', '节目名称': '心情好的日子', '声道语种': '韩语'}
{'形式': '电视剧', '类型': ['悬疑', '家庭'], '地区参数': '中国', '节目名称': '娘心', '声道语种': '汉语'}
{'形式': '电影', '类型': ['喜剧', '动画', '奇幻', '冒险'], '地区参数': '美国', '节目名称': '蓝精灵', '声道语种': '英语'}
{'形式': '电视剧', '类型': ['动画', '悬疑', '奇幻', '冒险'], '地区参数': '中国', '节目名称': '勇者大冒险', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['剧情', '爱情'], '地区参数': '台湾', '节目名称': 'A咖的路', '声道语种': '汉语'}
{'形式': '电视剧', '类型': ['儿童'], '地区参数'

{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '妈妈向前冲冲冲', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '寒武纪', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '大唐荣耀', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '红旗漫卷西风', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '元气拯救队', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '异能家庭', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '夺宝幸运星', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '迷', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '周末父母', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '一万公里的约定', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '超级飞侠', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '射雕英雄传', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '我住在这里的理由', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '决战江桥', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '天籁之战'

{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '警察故事4之简单任务', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '何所冬暖何所夏凉', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '逗逗迪迪爱唱歌', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '艾可魔法少女', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '泡芙小姐', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '小蜜蜂', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '贝瓦儿歌', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '快乐大本营', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '雷霆扫毒', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '口水三国', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '我是未来', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '国民大生活', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '果宝特攻三', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '镇魂街', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '

{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '猪猪侠超星萌宠二', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '平凡岁月', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '美味奇缘', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '29+1', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '吾儿可教', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '奇葩追梦', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '棒球英豪', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': 'CODE', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '功夫小兔侠', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '纳米核心', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '寻灵大冒险', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '一站到底', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '神兽金刚', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '秦时明月之万里长城', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '了不起的孩

...

{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '小龙大功夫第五季', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '热血尖兵', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '疑犯追踪', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '拜拜啦肉肉2016', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '追凶', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '疯狂时代2', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '擎天无影脚黄麒英', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '巾帼枭雄之谍血长天', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '熊出没之夏日连连看', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '阿爷厨房', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '米奇妙妙屋', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '真心想让你幸福', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '我的二哥二嫂', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '阿优之爆笑校园', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其

AttributeError: 'NoneType' object has no attribute 'xpath'

{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '新大头儿子和小头爸爸超能爸爸', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '斗龙战士4', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': 'Beautiful', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '好先生', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '忍冬艳蔷薇', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '挑战不可能', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '巴啦啦之飞越彩灵岛', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '戒魔人', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '星光灿烂', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '巴啦啦小魔仙之飞越彩灵堡', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '传世', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '飞哥战队', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '石敢当之雄峙天', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '探索·发现', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其

{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '虹色时光', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '超级飞侠三', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '一起又看流星雨', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '极速飞车', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '异星觉醒', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '第二次二十岁', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '我的女神女汉子', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '大唐玄奘', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '西游超级粉', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '新神探联盟', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '离婚律师', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '时尚迷搭', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '人间不拆', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '超级女特工', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '大地惊雷

{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '命运规划局', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '外公芳龄38', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '鲛珠传', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '东邪西毒', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '机智过人', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '当怪物来敲门', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '双刺', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '致命魔术', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '娇娇女上司', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '胖男孩快跑', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '黑猫警长', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '破冰船', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '我爱你么', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '长发公主', '声道语种': '其他'}
{'形式': '', '类型': '', '地区参数': '其他地区', '节目名称': '魔幻陀螺', '声道语种'

In [45]:
tfv = pd.DataFrame(tfv_list)
tfv.append(rest_prod)
tfv.reset_index(drop=True, inplace=True)

In [129]:
#首先处理语种，将汉语改为国语，其他的看情况修改
tfv["声道语种"].value_counts()
#tfv.loc[tfv["声道语种"]=="汉语", "声道语种"] = "国语"

国语    132
英语     88
日语     11
粤语      9
法语      4
韩语      3
其他      1
Name: 声道语种, dtype: int64

In [131]:
#再处理地区参数
tfv["地区参数"].value_counts()
#tfv.loc[tfv["地区参数"]=="其他国家", "地区参数"] = "其他地区"

内地      126
美国       69
英国       13
香港       13
日本       12
法国        5
其他地区      4
韩国        3
台湾        3
Name: 地区参数, dtype: int64

In [122]:
#根据形式来看有哪些节目没有形式和类型，然后手动添加
tfv["形式"].value_counts()
#tfv.loc[tfv["形式"]==""]
#tfv.loc[tfv["类型"]==""]

电影      155
电视剧      73
综艺        8
音乐        5
科学教育      5
生活服务      2
Name: 形式, dtype: int64

In [118]:
# tfv.loc[tfv["节目名称"].str.contains("举杯呵呵喝"), "地区参数"] = "中国"
# tfv.loc[tfv["节目名称"].str.contains("举杯呵呵喝"), "声道语种"] = "国语"
# tfv.loc[tfv["节目名称"].str.contains("举杯呵呵喝"), "形式"] = "综艺"
# tfv.loc[tfv["节目名称"].str.contains("举杯呵呵喝"), "类型"] = [['脱口秀','搞笑']]

In [108]:
# tfv.loc[tfv["节目名称"].str.contains("新红楼梦"), "地区参数"] = "中国"
# tfv.loc[tfv["节目名称"].str.contains("新红楼梦"), "声道语种"] = "国语"
# tfv.loc[tfv["节目名称"].str.contains("新红楼梦"), "形式"] = "电视剧"
# tfv.loc[tfv["节目名称"].str.contains("新红楼梦"), "类型"] = [['剧情','爱情','家庭','古装']]

In [107]:
# tfv.loc[tfv["节目名称"].str.contains("果宝特攻4之铠之魂"), "地区参数"] = "中国"
# tfv.loc[tfv["节目名称"].str.contains("果宝特攻4之铠之魂"), "声道语种"] = "国语"
# tfv.loc[tfv["节目名称"].str.contains("果宝特攻4之铠之魂"), "形式"] = "电视剧"
# tfv.loc[tfv["节目名称"].str.contains("果宝特攻4之铠之魂"), "类型"] = [['动画']]

In [120]:
# tfv.loc[tfv["节目名称"].str.contains("费加罗的婚礼"), "地区参数"] = "其他地区"
# tfv.loc[tfv["节目名称"].str.contains("费加罗的婚礼"), "声道语种"] = "英语"
# tfv.loc[tfv["节目名称"].str.contains("费加罗的婚礼"), "形式"] = "音乐"
# tfv.loc[tfv["节目名称"].str.contains("费加罗的婚礼"), "类型"] = [['Live']]

In [148]:
# new_all_program = pd.read_csv("all_program.csv", encoding="GB18030")
# #rest_prod["类型"] = ["[{}]".format(i) for i in rest_prod["类型"].values]
# new_program = tfv.append(rest_prod)
# new_program.reset_index(drop=True, inplace=True)
# new_all_program = all_program.append(new_program)
# new_all_program.reset_index(drop=True, inplace=True)

# 得到动画形式

In [16]:
#得到动画形式
def get_donghua_form(new_all_program):
    for i in new_all_program.loc[new_all_program["形式"] == "电视剧"]["节目名称"]:
        tag = eval(new_all_program.loc[new_all_program["节目名称"]== i, "类型"].values[0])
        if "动画" in tag:
            new_all_program.loc[new_all_program["节目名称"]== i, "形式"] = "动画"
    return new_all_program

In [18]:
new_all_program = get_donghua_form(tfv)

In [20]:
new_all_program.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2643 entries, 0 to 2642
Data columns (total 5 columns):
地区参数    2643 non-null object
声道语种    2643 non-null object
形式      2643 non-null object
类型      2643 non-null object
节目名称    2642 non-null object
dtypes: object(5)
memory usage: 103.3+ KB


In [21]:
new_all_program.to_csv("item_info_new.csv", index=False)